In [144]:
from bs4 import BeautifulSoup
import requests

In [145]:

ads_all = []
base_url = "https://www.bidbud.co.nz/search?category=0002-0356-0032-2273-&page="

page_num = 1

response = requests.get("https://www.bidbud.co.nz/search?category=0002-0356-0032-2273-&page=" + str(page_num))
html = response.content
soup = BeautifulSoup(html, "html.parser")
page_ads = soup.find_all('a', class_="listing_title")

ads_all.extend(page_ads)

while len(page_ads) > 0:
    page_num += 1
    response = requests.get("https://www.bidbud.co.nz/search?category=0002-0356-0032-2273-&page=" + str(page_num))
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    page_ads = soup.find_all('a', class_="listing_title")

    ads_all.extend(page_ads)

ads_hrefs = [a.get('href') for a in ads_all]

In [146]:
# Post clean schema perhaps?

dict_keys = ['ListingId', \
             'Title', \
             'Category', \
             'StartPrice', \
             'BuyNowPrice', \
             'StartDate', \
             'EndDate', \
             'IsFeatured', \
             'BiddersAndWatchers', \
             'MaxMidAmount', \
             'IsNew', \
             'Region', \
             'Suburb', \
             'BidCount', \
             'ViewCount', \
             'IsReserveMet', \
             'HasReserve', \
             'HasBuyNow', \
             'ReserveState', \
             'RelistedItemId', \
             'Subtitle', \
             'IsBuyNowOnly', \
             'TotalReviewCount', \
             'UnansweredQuestionCount', \
            ]

listing_dicts = [dict.fromkeys(dict_keys) for i in range(len(ads_all) + 1)]


In [168]:
listing_base_url = "https://www.bidbud.co.nz/"

response = requests.get("https://www.bidbud.co.nz/" + ads_hrefs[2])
html = response.content
soup = BeautifulSoup(html, "html.parser")

In [167]:
ads_hrefs[2]

'/4756489268'

In [161]:
title = soup.find('h2')

item_info = soup.find('div', id="item_info") # Listing start/end, prices, watches

description = soup.find('div', id='description') # Description element 

listing_attributes = description.find('table', id='listingattributes') # table with memory, HDD, screen, cores. Child of description
attr_rows = listing_attributes.findAll('tr')
attr_headers = []
attr_values = []
for row in attr_rows:
    header = row.find('th').text.strip()
    value = row.find('td').text.strip().strip(':')
    attr_headers.append(header)
    attr_values.append(value)

description_paragraphs = description.findAll('p')
description_paragraphs = [p.text for p in description_paragraphs]

In [162]:
seller_information = description.find('div', class_='panel-body')

In [163]:
member_latest_listings_href = seller_information.find('a', title="View member's listings").get('href')
member_username = seller_information.find('a', title="View member's listings").text

member_feedback_href = seller_information.find('a', title="View member's feedback").get('href')
member_feedback_score = seller_information.find('a', title="View member's feedback").text

seller_information_string = seller_information.text.strip() # Will need to clean this 


In [164]:
view_count = soup.find('span', id="view_count").text

In [165]:
listing_dict = dict()

listing_dict['ListingId'] = ads_hrefs[0].strip('/')
listing_dict['Title'] = title

attrs_dict = dict(zip(attr_headers, attr_values))
listing_dict.update(attrs_dict)

listing_dict['Description'] = description_paragraphs

listing_dict['MemberFeedbackUrl'] = member_feedback_href
listing_dict['MemberFeedbackScore'] = member_feedback_score
listing_dict['SellerInformationString'] = seller_information_string

listing_dict['ViewCount'] = view_count

listing_dict['ItemInfo'] = item_info.text.strip()



In [166]:
listing_dict

{'ListingId': '4760235612',
 'Title': <h2>Macbook Air 13 inch 2017</h2>,
 'Memory:': '8 to 15 GB',
 'Hard Drive Size:': 'Under 160 GB',
 'Screen Size:': '13"',
 'Cores:': '5',
 'Description': ['MacBook Air 13 Inch, 2017 1, 8 GHz  corei5, 8 GB .\nWorks well when plugged to power  but the battery is not great and will only hold charge for a couple of hours.',
  'NO POSTING ! BUYER MUST PICK UP.'],
 'MemberFeedbackUrl': 'https://www.bidbud.co.nz/members/feedback/568396',
 'MemberFeedbackScore': '196',
 'SellerInformationString': 'loopy9999\xa0(196\xa0\xa0)Member for 20 years99.0% positive feedbackLocated in Cambridge, WaikatoPayment methods : Cash, NZ Bank Deposit',
 'ViewCount': '75 (viewed on average every 30 minutes )',
 'ItemInfo': 'Listed on 13 Jun 2024. Closes Tue 18 Jun, 4:46pm — 3 daysStart price $100.00 reserve not met 4 people added this to their watchlist'}